**기본설정**

In [ ]:
!nvidia-smi || true
!python -V
!pip -q install kaggle --upgrade

import os, random, time, shutil, zipfile, gc
from pathlib import Path

import numpy as np
from tqdm.auto import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image

# GPU 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 시드 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device.type == "cuda":
    torch.cuda.manual_seed_all(SEED)


**데이터 다운로드**

In [ ]:
from google.colab import files

print("👉 kaggle.json을 업로드하세요 (Kaggle > Account > Create New API Token)")
uploaded = files.upload()  # kaggle.json 업로드

os.makedirs(Path.home()/".kaggle", exist_ok=True)
shutil.move("kaggle.json", str(Path.home()/".kaggle/kaggle.json"))
os.chmod(str(Path.home()/".kaggle/kaggle.json"), 0o600)

# 데이터 다운로드
!kaggle competitions download -c dogs-vs-cats -f train.zip -p /content
!ls -lh /content/train.zip


In [ ]:
DATA_ROOT = Path("/content/dogs_v_cats")
RAW_DIR = DATA_ROOT / "raw_train"
RAW_DIR.mkdir(parents=True, exist_ok=True)

# 압축 해제 (이미 풀려있으면 다시 안 풀어도 됨)
if not any(RAW_DIR.iterdir()):
    t0 = time.time()
    with zipfile.ZipFile("/content/train.zip") as z:
        z.extractall(RAW_DIR)
    print("압축 해제 완료:", round(time.time()-t0, 2), "초")

# 하위 폴더까지 다 뒤져서 이미지 찾기
all_imgs = sorted([p for p in RAW_DIR.rglob("*") if p.suffix.lower() in [".jpg", ".jpeg", ".png"]])

print("총 이미지:", len(all_imgs))
if len(all_imgs) > 0:
    print("예시:", all_imgs[0].name, "…", all_imgs[-1].name)
else:
    raise RuntimeError("⚠️ 이미지가 발견되지 않았습니다. 경로 확인 필요!")


In [ ]:
class DogsCatsDataset(Dataset):
    def __init__(self, files, transform=None):
        self.files = files
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def _label_from_name(self, path: Path):
        name = path.name.lower()
        if name.startswith("cat"):
            return 0
        elif name.startswith("dog"):
            return 1
        if "cat" in name: return 0
        if "dog" in name: return 1
        raise ValueError(f"라벨 알 수 없음: {path.name}")

    def __getitem__(self, idx):
        path = self.files[idx]
        label = self._label_from_name(path)
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label


In [ ]:
# 8:2 분할
random.shuffle(all_imgs)
split_idx = int(len(all_imgs) * 0.8)
train_files = all_imgs[:split_idx]
valid_files = all_imgs[split_idx:]

print(f"Train: {len(train_files)}, Valid: {len(valid_files)}")

IMG_SIZE = 224

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

valid_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

train_ds = DogsCatsDataset(train_files, transform=train_tfms)
valid_ds = DogsCatsDataset(valid_files, transform=valid_tfms)

pin_memory = (device.type == "cuda")
num_workers = min(4, os.cpu_count() or 1)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True,
                          num_workers=num_workers, pin_memory=pin_memory)
valid_loader = DataLoader(valid_ds, batch_size=128, shuffle=False,
                          num_workers=num_workers, pin_memory=pin_memory)


In [ ]:
from torchvision.models import resnet18, ResNet18_Weights

weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5)

use_amp = (device.type == "cuda")
if use_amp:
    scaler = torch.amp.GradScaler(device="cuda")
else:
    scaler = torch.amp.GradScaler(enabled=False)

print("AMP 사용:", use_amp)


In [ ]:
def accuracy(outputs, targets):
    preds = outputs.argmax(dim=1)
    return (preds == targets).float().mean().item()

EPOCHS = 5
best_acc = -1.0
save_path = "/content/resnet18_dogs_cats_best.pt"

for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss, train_acc = 0.0, 0.0

    pbar = tqdm(total=len(train_loader), desc=f"[Epoch {epoch}/{EPOCHS}] Train", unit="batch")
    for i, (imgs, labels) in enumerate(train_loader, start=1):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad(set_to_none=True)

        if use_amp:
            with torch.amp.autocast("cuda"):
                outputs = model(imgs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        train_loss += loss.item()
        train_acc  += accuracy(outputs, labels)
        pbar.set_postfix(loss=f"{train_loss/i:.4f}", acc=f"{(train_acc/i)*100:.2f}%")
        pbar.update(1)
    pbar.close()

    # Validation
    model.eval()
    val_loss, val_acc = 0.0, 0.0
    with torch.no_grad():
        pbar_v = tqdm(total=len(valid_loader), desc=f"[Epoch {epoch}/{EPOCHS}] Valid", unit="batch")
        for j, (imgs, labels) in enumerate(valid_loader, start=1):
            imgs, labels = imgs.to(device), labels.to(device)
            if use_amp:
                with torch.amp.autocast("cuda"):
                    outputs = model(imgs)
                    loss = criterion(outputs, labels)
            else:
                outputs = model(imgs)
                loss = criterion(outputs, labels)
            val_loss += loss.item()
            val_acc  += accuracy(outputs, labels)
            pbar_v.set_postfix(loss=f"{val_loss/j:.4f}", acc=f"{(val_acc/j)*100:.2f}%")
            pbar_v.update(1)
        pbar_v.close()

    print(f"Epoch {epoch}: Train acc={train_acc/len(train_loader)*100:.2f}%, Val acc={val_acc/len(valid_loader)*100:.2f}%")

    scheduler.step()

    if val_acc/len(valid_loader) > best_acc:
        best_acc = val_acc/len(valid_loader)
        torch.save(model.state_dict(), save_path)
        print(f"✅ Best model saved (val_acc={best_acc*100:.2f}%)")

In [ ]:
# 모델 로드
model.load_state_dict(torch.load(save_path, map_location=device))
model.eval()

label2name = {0: "cat", 1: "dog"}
idxs = np.random.choice(len(valid_ds), size=5, replace=False)

with torch.no_grad():
    for idx in idxs:
        img_path = valid_files[idx]
        img = Image.open(img_path).convert("RGB")
        inp = valid_tfms(img).unsqueeze(0).to(device)
        with torch.amp.autocast("cuda") if use_amp else torch.no_grad():
            out = model(inp)
        pred = out.argmax(dim=1).item()
        print(f"[예측: {label2name[pred]}] → {img_path.name}")
        display(img.resize((256,256)))


In [ ]:
# 최종 Validation 정확도 한 번 더 계산
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in valid_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_acc = correct / total * 100
print(f"최종 Validation Accuracy: {val_acc:.2f}%")
